In [16]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
#import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta

In [17]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [18]:
import pickle
with open('dict_corrected_DH.obj', 'rb') as handle:
    dict_corrected_DH = pickle.load(handle)
with open('geocoded_locs.obj', 'rb') as handle:
    geocoded_locs = pickle.load(handle)

with open('corpus_mails_DH.obj', 'rb') as handle:
    corpus_mails_DH = pickle.load(handle)

with open('dict_noms_mails_debut_fin.obj', 'rb') as handle:
    dict_noms_mails_debut_fin = pickle.load(handle)
    

with open('liste_de_tous_les_mails.obj', 'rb') as handle:
    liste_de_tous_les_mails = pickle.load(handle)


In [19]:

    
print str(dict_corrected_DH)[:1000],"\n"
print str(geocoded_locs)[:1000],"\n"
print str(corpus_mails_DH)[:1000],"\n"
print str(dict_noms_mails_debut_fin)[:1000],"\n"
print str(liste_de_tous_les_mails)[:1000],"\n"


{'googlegroups.com': {u'geometry': {u'location': {u'lat': '43.8424986', u'lng': '4.354806'}}}, 'lecnam.net': {u'geometry': {u'location': {u'lat': '43.8424986', u'lng': '4.354806'}}}, 'jiscmail.ac.uk': {u'geometry': {u'location': {u'lat': '43.8424986', u'lng': '4.354806'}}}, 'design.ulaval.ca': {u'geometry': {u'location': {u'lat': 46.812707, u'lng': -71.225985}, u'viewport': {u'northeast': {u'lat': 46.8140559802915, u'lng': -71.2246360197085}, u'southwest': {u'lat': 46.8113580197085, u'lng': -71.2273339802915}}, u'location_type': u'GEOMETRIC_CENTER'}, u'address_components': [{u'long_name': u'Qu\xe9bec', u'types': [u'locality', u'political'], u'short_name': u'Qu\xe9bec'}, {u'long_name': u'Qu\xe9bec', u'types': [u'administrative_area_level_1', u'political'], u'short_name': u'QC'}, {u'long_name': u'Canada', u'types': [u'country', u'political'], u'short_name': u'CA'}, {u'long_name': u'G1K 3G8', u'types': [u'postal_code'], u'short_name': u'G1K 3G8'}], u'place_id': u'ChIJDWgVp3GWuEwRKa5K0WLTY

In [20]:
from datetime import timedelta
import datetime
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
title = "DH_studies_map"
my_nodesdict = {}
my_nodes= []
my_edges = []


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'



###FOR NODES
print len(dict_noms_mails_debut_fin)
print dict_noms_mails_debut_fin['Claire Clivaz']
#claire.clivaz@unil.ch

167
[['claire.clivaz@unil.ch', datetime.datetime(2013, 7, 9, 15, 28, 26, tzinfo=tzoffset(None, 7200)), datetime.datetime(2014, 7, 12, 12, 31, 28, tzinfo=tzoffset(None, 7200))], ['claire.clivaz@sib.swiss', datetime.datetime(2016, 7, 20, 16, 54, 26, tzinfo=tzoffset(None, 7200)), datetime.datetime(2016, 7, 29, 13, 21, 16, tzinfo=tzoffset(None, 7200))]]


In [21]:









for name in dict_noms_mails_debut_fin:
    
    for mail_array in dict_noms_mails_debut_fin[name]:
        print mail_array[0]
        mail_address=mail_array[0]
        start= mail_array[1]
        end= mail_array[2]
        mail_address_suffix=mail_address.split("@",1)[1]
        print mail_address_suffix
        lat=dict_corrected_DH[mail_address_suffix.lower()]["geometry"]["location"]["lat"]
        lng=dict_corrected_DH[mail_address_suffix.lower()]["geometry"]["location"]["lng"]
        print  lat,lng
        
        print "-------"
         
        try: 
            #print start
            #print end
            node = {
                        "id" : mail_address,
                        "name" : name,
                       "lat" : float(lat),
                       "lng" : float(lng),
                       "weight" : float(4),
                        #"start" : start.isoformat(),

                       #"end" : (end+timedelta(days=1)).isoformat()
                        }
        except AttributeError as e:
            print "STARTSTART",start
            #print datetime.datetime(start[:-5])
            print e

            node = {
                        "id" : mail_address,
                        "name" : name,
                       "lat" : float(lat),
                       "lng" : float(lng),
                       "weight" : float(4),
                        #"start" : datetime.datetime(2012, 3, 2, 15, 19, 10).isoformat(),

                       #"end" : datetime.datetime(2018, 3, 2, 15, 19, 10).isoformat()
                        }
            print node
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
    
print len(my_nodesdict)
    
        
    
    
    

pierre.mounier@openedition.org
openedition.org
43.8424986 4.354806
-------
pierre.mounier@ehess.fr
ehess.fr
48.850174 2.3268487
-------
aude.da-cruz-lima@mae.u-paris10.fr
mae.u-paris10.fr
48.9041663 2.2098627
-------
thierry.poibeau@ens.fr
ens.fr
48.8422058 2.3451689
-------
delegue.general@adbu.fr
adbu.fr
43.8424986 4.354806
-------
dominique.stutzmann@irht.cnrs.fr
irht.cnrs.fr
48.8664818 2.2954043
-------
martin.grandjean@unil.ch
unil.ch
46.5210895 6.5801606
-------
Nicolas.Larrousse@huma-num.fr
huma-num.fr
48.850142 2.3269827
-------
judith.hannoun@univ-amu.fr
univ-amu.fr
43.293621 5.358066
-------
ottaviano.nancy@gmail.com
gmail.com
43.8424986 4.354806
-------
nadine.dardenne@cnrs.fr
cnrs.fr
45.203453 5.700236
-------
colette.cadiou@irstea.fr
irstea.fr
45.7627097 3.1110909
-------
yamina.bensaadoune@univ-rouen.fr
univ-rouen.fr
49.4583047 1.0688892
-------
francoise.paquienseguy@sciencespo-lyon.fr
sciencespo-lyon.fr
45.7470623 4.8357091
-------
cecile.rodrigues@cnrs.fr
cnrs.fr
45.20

In [22]:
print len(my_nodesdict)

173


In [40]:
import random
def my_edges_list_singled() :
    my_edges=[]
    extra_nodes_non_autor_list=[]
    for mail in corpus_mails_DH:
        #print mail
         #print  corpus_mails_DH[mail]
        if "liste_CCs" in corpus_mails_DH[mail]:
            for maild in corpus_mails_DH[mail]["liste_CCs"] :
                if maild not in dict_noms_mails_debut_fin:
                    if maild not in extra_nodes_non_autor_list:
                        print "OHOH NO, NO maild in liste_mail", maild
                        extra_nodes_non_autor_list.append(maild)
                        print corpus_mails_DH[mail]["sujet_du_message"]
                        if "sujet_du_message" in corpus_mails_DH[mail]:
                            edge = {
                        "id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"])+" "+corpus_mails_DH[mail]["mail_auteur"]+maild+str(float(random.random())),
                        "source" : corpus_mails_DH[mail]["mail_auteur"],
                        "target" : maild,
                        "weight" : float(2),
                        "name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])
                    }
                        else:
                            edge = {
                        "id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"])+" "+corpus_mails_DH[mail]["mail_auteur"]+maild+str(float(random.random())),
                        "source" : corpus_mails_DH[mail]["mail_auteur"],
                        "target" : maild,
                        "weight" : float(2),
                       
                    }
                            
                my_edges.append({ "data" : edge })

        for maild in corpus_mails_DH[mail]["liste_mails_dests"] :
            if maild not in dict_noms_mails_debut_fin:
                    if maild not in extra_nodes_non_autor_list:
                        #print "OHOH NO, NO maild in liste_mail", maild
                        extra_nodes_non_autor_list.append(maild)
            
                        if "sujet_du_message" in corpus_mails_DH[mail]:
                            edge = {
                        "id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"])+" "+corpus_mails_DH[mail]["mail_auteur"]+maild+str(float(random.random())),
                        "source" : corpus_mails_DH[mail]["mail_auteur"],
                        "target" : maild,
                        "weight" : float(2),
                        "name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])
                    }
                        else:
                            edge = {
                        "id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"])+" "+corpus_mails_DH[mail]["mail_auteur"]+maild+str(float(random.random())),
                        "source" : corpus_mails_DH[mail]["mail_auteur"],
                        "target" : maild,
                        "weight" : float(2),
                       
                    }
            
            my_edges.append({ "data" : edge })



    #print len(my_edges)
    #print extra_nodes_non_autor_list
    #print len(extra_nodes_non_autor_list)
    return [my_edges,extra_nodes_non_autor_list]

my_edges_singled=my_edges_list_singled()[0]
extra_nodes=my_edges_list_singled()[1]

OHOH NO, NO maild in liste_mail marjorie.burghart@ehess.fr
: [dh] recherche reproductible et dh (was: collo que sur les digital humanities : appel à contribution )
OHOH NO, NO maild in liste_mail jean-daniel.fekete@inria.fr
: [dh] recherche reproductible et dh (was: collo que sur les digital humanities : appel à contribution )
OHOH NO, NO maild in liste_mail claire.clivaz@unil.ch
: re: [dh] thatcamp saint-malo - association francophone des digital humanities
OHOH NO, NO maild in liste_mail frederic@clavert.net
: re: [dh] thatcamp saint-malo - association francophone des digital humanities
OHOH NO, NO maild in liste_mail francesca.frontini@univ-montp3.fr
: [dh] second cfp: workshop on language, ontology, terminology and knowledge structures (lotks - 2017)
OHOH NO, NO maild in liste_mail humanum-veille@listes.huma-num.fr
: [dh] fwd: comité programme semweb.pro 2017
OHOH NO, NO maild in liste_mail thibault.clerice@enc-sorbonne.fr
: [dh] candidatures aux masters humanités numériques et rec

In [41]:
#print extra_nodes


for mail in extra_nodes:
    
        print mail
        
#        start= mail_array[1]
#        end= mail_array[2]
        mail_address_suffix=mail.split("@",1)[1]
        print mail_address_suffix
        if mail_address_suffix in dict_corrected_DH  :
            
            lat=dict_corrected_DH[mail_address_suffix]["geometry"]["location"]["lat"]
            lng=dict_corrected_DH[mail_address_suffix]["geometry"]["location"]["lng"]
            node = {
                        "id" : mail,
                        "name" : mail,
                       "lat" : float(lat),
                       "lng" : float(lng),
                       "weight" : float(4),

                        }
        else: 
            
        
            node = {
                        "id" : mail,
                        "name" : mail,
                       "weight" : float(4),
                        
                        }
        
            print node
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
    
print len(my_nodesdict)
    
        
    
    
    


dh@groupes.renater.fr
groupes.renater.fr
legram@yahoogroupes.fr
yahoogroupes.fr
culture.numerique@ml.free.fr
ml.free.fr
listesocius@groups.openedition.org
groups.openedition.org
stephane.pouyllau@cnrs.fr
cnrs.fr
exploreat@oeaw.ac.at
oeaw.ac.at
humanist@lists.digitalhumanities.org
lists.digitalhumanities.org
globaloutlookdh-l@uleth.ca
uleth.ca
digitalclassicist@jiscmail.ac.uk
jiscmail.ac.uk
dm-l@uleth.ca
uleth.ca
dhd@mailman.rrz.uni-hamburg.de
mailman.rrz.uni-hamburg.de
aiucd-l@humnet.unipi.it
humnet.unipi.it
air-l@aoir.org
aoir.org
institute@lists.uvic.ca
lists.uvic.ca
tei-l@listserv.brown.edu
listserv.brown.edu
humanist-l@uleth.ca
uleth.ca
centernet@lists.digitalhumanities.org
lists.digitalhumanities.org
dhcarolina@listserv.unc.edu
listserv.unc.edu
humanidadesdigitais@gmail.com
gmail.com
website@hastac.org
hastac.org
southasia-dh@lists.globaloutlookdh.org
lists.globaloutlookdh.org
air-l@listserv.aoir.org
listserv.aoir.org
humanisticadh@gmail.com
gmail.com
marjorie.burghart@ehess.fr
eh

In [42]:
my_edges=my_edges_singled
print len(my_edges)
print my_edges

526
[{'data': {'source': 'jorge.fins@univ-tours.fr', 'name': u': re: [dh] prestation de d\xe9veloppement sous drupal', 'id': u'./DHFRsample/2017-06/msg00011.html jorge.fins@univ-tours.frdh@groupes.renater.fr0.119592432592', 'weight': 2.0, 'target': 'dh@groupes.renater.fr'}}, {'data': {'source': 'dumouchelsuzanne@yahoo.fr', 'name': u': [dh] conf\xe9rence de marcello vitali-rosati [s\xe9minaire ephn]', 'id': u'./DHFRsample/2018-03/msg00011.html dumouchelsuzanne@yahoo.frlegram@yahoogroupes.fr0.819090356692', 'weight': 2.0, 'target': 'legram@yahoogroupes.fr'}}, {'data': {'source': 'dumouchelsuzanne@yahoo.fr', 'name': u': [dh] conf\xe9rence de marcello vitali-rosati [s\xe9minaire ephn]', 'id': u'./DHFRsample/2018-03/msg00011.html dumouchelsuzanne@yahoo.frculture.numerique@ml.free.fr0.656364203005', 'weight': 2.0, 'target': 'culture.numerique@ml.free.fr'}}, {'data': {'source': 'dumouchelsuzanne@yahoo.fr', 'name': u': [dh] conf\xe9rence de marcello vitali-rosati [s\xe9minaire ephn]', 'id': u'

In [83]:
def my_edges_list_by_source_dest() :
    my_edges=[]

    for mail in corpus_mails_DH:
        #print mail
         #print  corpus_mails_DH[mail]
        if "liste_CCs" in corpus_mails_DH[mail]:
            for maild in corpus_mails_DH[mail]["liste_CCs"] :
                #print unicode(corpus_mails_DH[mail]["sujet_du_message"])
                edge = {
                    #"id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"]),
                    "source" : corpus_mails_DH[mail]["mail_auteur"],
                    "target" : maild,
                    "weight" : float(2),
                    #"name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])+" "+
                }
                my_edges.append({ "data" : edge })

        for maild in corpus_mails_DH[mail]["liste_mails_dests"] :
            edge = {
                #"id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"]),
                "source" : corpus_mails_DH[mail]["mail_auteur"],
                "target" : maild,
                "weight" : float(5),
                #"name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])
            }
            my_edges.append({ "data" : edge })



    print len(my_edges)
    return my_edges

my_edges_duped=my_edges_list_by_source_dest()

527


In [92]:
def my_edges_list_with_subject() :
    my_edges=[]

    for mail in corpus_mails_DH:
        #print mail
         #print  corpus_mails_DH[mail]
        if "liste_CCs" in corpus_mails_DH[mail]:
            for maild in corpus_mails_DH[mail]["liste_CCs"] :
                #print unicode(corpus_mails_DH[mail]["sujet_du_message"])
                edge = {
                    #"id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"]),
                    "source" : corpus_mails_DH[mail]["mail_auteur"],
                    "target" : maild,
                    #"weight" : float(2),
                    "name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])
                }
                my_edges.append({ "data" : edge })

        for maild in corpus_mails_DH[mail]["liste_mails_dests"] :
            edge = {
                #"id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"]),
                "source" : corpus_mails_DH[mail]["mail_auteur"],
                "target" : maild,
                #"weight" : float(5),
                "name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])
            }
            my_edges.append({ "data" : edge })



    print len(my_edges)

    return my_edges

my_edges_duped_with_subject=my_edges_list_with_subject()

527


In [85]:


#WE HAVE TO TEST THE EDGES, BUT IT IS AN ARRAY OF DICTS,SO WE NEED TO FLATTEN THEM TO COUNT THEM SIMPLY

import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)




In [86]:
def list_flattener_and_finder(edges_list_singled,edges_list_duped):
    from collections import Counter
    import ast
    my_edges_replacement_deduped=[]
    receiver=[]
    for i,dic in enumerate(edges_list_duped):
        #print my_edges[i]
        dicto=flatten(edges_list_duped[i])
    #    print dicto
        receiver.append(str(dicto))

    #print receiver
    CREE= Counter(receiver).iteritems()
    total=0
    dups=0
    dups_cnt=0
    
    print edges_list_singled[:1]
    for key, value in CREE:
        #print key
        #print value
        keyB={'data':{}}
        total+=1
        
        #print ast.literal_eval(key)["data_source"]
        #print ast.literal_eval(key)["data_target"]        
        keyB={'data':{"source":ast.literal_eval(key)["data_source"],"target": ast.literal_eval(key)["data_target"]}}
        #print "SUP",key, value
        dups+=value
        dups_cnt+=1
        #print ">>",keyB
        #print ast.literal_eval(key)["data_weight"]    
        keyB["data"]["weight"]= float(value*float(ast.literal_eval(key)["data_weight"]))
        #print "KEYB____",keyB
        my_edges_replacement_deduped.append(keyB)


            





    print "total",total

    print "dups",dups
    print "dups_cnt",dups_cnt
    return my_edges_replacement_deduped

listor=list_flattener_and_finder(my_edges,my_edges_duped)
print listor

[]
total 368
dups 527
dups_cnt 368
[{'data': {'source': 'e.salvatori@mediev.unipi.it', 'target': 'dh@groupes.renater.fr', 'weight': 5.0}}, {'data': {'source': 'delegue.general@adbu.fr', 'target': 'dh@groupes.renater.fr', 'weight': 5.0}}, {'data': {'source': 'graham.ranger@univ-avignon.fr', 'target': 'parislinguists@yahoogroupes.fr', 'weight': 5.0}}, {'data': {'source': 'Guillaume.Blum@design.ulaval.ca', 'target': 'docs-ri@yahoogroupes.fr', 'weight': 5.0}}, {'data': {'source': 'jahjah.marc@gmail.com', 'target': 'dh@groupes.renater.fr', 'weight': 5.0}}, {'data': {'source': 'gpansu@gmail.com', 'target': 'dh@groupes.renater.fr', 'weight': 5.0}}, {'data': {'source': 'eric.kergosien@univ-lille3.fr', 'target': 'liste-proml@lri.fr', 'weight': 5.0}}, {'data': {'source': 'emmanuelle.morlock@gmail.com', 'target': 'dh@groupes.renater.fr', 'weight': 5.0}}, {'data': {'source': 'claire.lemercier@sciencespo.fr', 'target': 'dh@groupes.renater.fr', 'weight': 5.0}}, {'data': {'source': 'yamina.bensaadoun

In [87]:
res=dups/dups_cnt



NameError: name 'dups' is not defined

In [88]:
my_edges=listor

In [51]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)
my_nodesdictList = []
for key, value in my_nodesdict.iteritems():
    
    my_nodesdictList.append(value)



ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'ZSzPocQOLmQolF_1Y7f9KJxj2OKsSX_ydkcZyKdfrYb', u'userId': u'DaofbRenqYpqqNmtz'}}


In [44]:
#print my_edges
list_nodes_id=[] 
for nodes in  my_nodesdictList:
    if nodes['data']['id'] not in list_nodes_id:
        list_nodes_id.append(nodes['data']['id'])
    
print len(list_nodes_id)

print list_nodes_id
for edge in my_edges:
    source= edge['data']['source']
    target= edge['data']['target']
    if source not in list_nodes_id:
        print "NO!"
        print source
    if target not in list_nodes_id:
        print "NO2!"
        print target


279
['infoling@listserv.rediris.es', 'ahdig@googlegroups.com', 'digitalclassicist@jiscmail.ac.uk', 'diffusion@listes.ancmsp.com', 'rbdd@services.cnrs.fr', 'madics-adoc@listes.univ-lyon2.fr', 'frederic@clavert.net', 'audrey.baneyx@sciencespo.fr', 'vial.stephane@gmail.com', 'marie.e.lescasse@gmail.com', 'laurence.rageot@univ-tours.fr', 'clemence.jacquot@gmail.com', 'dhcarolina@listserv.unc.edu', 'globaloutlookdh-l@uleth.ca', 'aurelie.olivesi@univ-lyon1.fr', 'michel.bernard@univ-paris3.fr', 'laurent.romary@inria.fr', 'valerie.beaugiraud@ens-lyon.fr', 'bull-i3@irit.fr', 'nep-his@lists.repec.org', 'ottaviano.nancy@gmail.com', 'yamina.bensaadoune@univ-rouen.fr', 'alexandre.hocquet@univ-lorraine.fr', 'beatrice.markhoff@univ-tours.fr', 'c.schoech@gmail.com', 'm.bourgatte@icp.fr', 'dominique.stutzmann@irht.cnrs.fr', 'caroline.cance@univ-orleans.fr', 'jerome.darmont@univ-lyon2.fr', 'pascal.cristofoli@ehess.fr', 'recherche-design@listes.univ-paris1.fr', 'Mathieu.Andro@versailles.inra.fr', 'Guilla

In [46]:
TOPOGRAM_URL = "http://localhost:3000"

In [49]:
TOPOGRAM_URL = "https://app.topogram.io"

In [53]:
title = "alphaDH19"
create_topogram(title, my_nodesdictList, my_edges)

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


Creating topogram 'alphaDH19'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'alphaDH19', u'userId': u'DaofbRenqYpqqNmtz', u'_id': u'9mQYrf4QdiEcnaf96', u'slug': u'alphadh19', u'createdAt': u'2018-04-25T19:24:44.278Z'}, u'statusCode': 200}
topogram ID : 9mQYrf4QdiEcnaf96
{u'status': u'success', 'status_code': 200, u'data': [u'Gd7EeHGEFAAogxoGt', u'8yGaD9gZCXvx4ADGp', u'Jz387C28u9AxQP5FT', u'daxAKStQg4d6tjnYA', u'NBiqysaCq3hGe3oPw', u'WKWbrtPukWTuqJRBe', u'KmuDSpT5LhrEi554u', u'uzzgEtuh8TFKw7yoi', u'ks829ZPCwhDvsnbob', u'EXtjF6SBtHx4vSoLo', u'GQ7ASvpWiRhvGQjGp', u'Ks4LRTBpZPcwTyKKt', u'RsXpvDWXWF4HhdSSz', u'ZE8CP2d6sAQJ8wuWe', u'yPvLbaNNdZyu7EHEt', u'gKe42zCjw4ztyjTTL', u'4AR6MJkkKZezqDFry', u'7dZ2ixkBakik8iAr9', u'7KXaFsasHPY7PmYFw', u'DAdxt2D63T5E9y7E9', u'3i6uDB67xvKSBSfSs', u'MrNSPZTTcmtZmWFeo', u'TBAzK4Xg8qhHtJAD8', u'tkTasQc7WMH5bzweT', u'TcC7a4CWmRvwKcS2S', u'uZq5LJu7TnLEDGe3p', u'abQehaoom5nQPG7jN', u'rZGA3gCaGT